In [1]:
!pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.5 MB 4.5 MB/s 


In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
!ls '/gdrive/MyDrive/0. 인공지능 학습/아이펠 대전 3기/2. Exploration 수업/코드/15강'

ex1.ipynb	 project15.ipynb  usersha1-artmbid-artname-plays.tsv
mbox_sha1sum.py  ratings.dat	  usersha1-profile.tsv
movies.dat	 users.dat


In [5]:
import numpy as np
import pandas as pd
import scipy
from scipy.sparse import csr_matrix

# import implicit
from implicit.als import AlternatingLeastSquares
import os



# 1. 전처리

In [6]:
rating_file_path = '/gdrive/MyDrive/0. 인공지능 학습/아이펠 대전 3기/2. Exploration 수업/코드/15강/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head(5)

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
print("마지막 유저 아이디 번호 :", ratings['user_id'].max())
print("마지막 영화 아이디 번호 :", ratings['movie_id'].max())

마지막 유저 아이디 번호 : 6040
마지막 영화 아이디 번호 : 3952


In [8]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [9]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [10]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [11]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path = '/gdrive/MyDrive/0. 인공지능 학습/아이펠 대전 3기/2. Exploration 수업/코드/15강/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.tail(5)

,movie_id,title,genre
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


# 2. 분석
- ratings에 있는 유니크한 영화 개수
- ratings에 있는 유니크한 사용자 수
- 가장 인기 있는 영화 30개(인기순)

In [12]:
ratings['movie_id'].nunique()

3628

In [13]:
ratings['user_id'].nunique()

6039

In [14]:
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

# 3. 선호하는 영화 5개 ratings에 추가
- 기존 데이터 프레임의 마지막 유저 아이디 번호 : 6040
- 기존 데이터 프레임의 마지막 영화 아이디 번호 : 3952

In [15]:
my_favorite = [3953 , 3954 ,3955 ,3956 ,3957]
my_playlist = pd.DataFrame({'user_id': [6041]*5, 'movie_id': my_favorite, 'counts': [30]*5, 'timestamp': [956715569.0]*5})

if not ratings.isin({'user_id':[6041]})['user_id'].any():
    ratings = ratings.append(my_playlist)

ratings.tail(10)

,user_id,movie_id,counts,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,6041,3953,30,956715569.0
1,6041,3954,30,956715569.0
2,6041,3955,30,956715569.0
3,6041,3956,30,956715569.0
4,6041,3957,30,956715569.0


In [16]:
df = pd.DataFrame({'movie_id': [3953 , 3954 ,3955 ,3956 ,3957],
                   'title': ['amollang(2022)', 'amollang(2023)', 'amollang(2024)', 'amollang(2025)', 'amollang(2026)'],
                   'genre': ['Comedy', 'Comedy', 'Comedy', 'Comedy', 'Comedy']})
df

,movie_id,title,genre
0,3953,amollang(2022),Comedy
1,3954,amollang(2023),Comedy
2,3955,amollang(2024),Comedy
3,3956,amollang(2025),Comedy
4,3957,amollang(2026),Comedy


In [17]:
movies = movies.append(df, ignore_index = True)
movies.tail(10)

,movie_id,title,genre
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller
3883,3953,amollang(2022),Comedy
3884,3954,amollang(2023),Comedy
3885,3955,amollang(2024),Comedy
3886,3956,amollang(2025),Comedy
3887,3957,amollang(2026),Comedy


In [18]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [19]:
print(user_to_idx[6041])
print(movie_to_idx[3957])

6039
3632


In [20]:
temp_user_ratings = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_ratings) == len(ratings):
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_ratings
else:
    print('user_id column indexing Fail!!')

temp_movie_ratings = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_ratings) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_ratings
else:
    print('movie_id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,counts,timestamp
0,0,0,5,978300760.0
1,0,1,3,978302109.0
2,0,2,3,978301968.0
3,0,3,4,978300275.0
4,0,4,5,978824291.0
...,...,...,...,...
0,6039,3628,30,956715569.0
1,6039,3629,30,956715569.0
2,6039,3630,30,956715569.0
3,6039,3631,30,956715569.0


# 4. CSR matrix 생성

In [21]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

print(num_user, num_movie)

6040 3633


In [22]:
csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape=(num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5. AlternatingLeastSquares 모델 훈련

In [23]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [24]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [25]:
# 모델 훈련
als_model.fit(csr_data)

  0%|          | 0/15 [00:00<?, ?it/s]

# 6. 선호도 예측

In [26]:
import random

def choose_random_user(df_ratings):
    random_index = random.randint(0, len(df_ratings))
    choosed_user = df_ratings.iloc[random_index]
    user_id = int(choosed_user[0])

    return user_id

def choose_random_movie(df_movies):
    random_index = random.randint(0, len(df_movies))
    choosed_movie = df_movies.iloc[random_index]
    movie_id = choosed_movie[0]
    title = choosed_movie[1]
    genre = choosed_movie[2]

    return movie_id, title, genre

In [27]:
user_id = 6041
movie_id = 3957
choosed_movie = movies.loc[movies['movie_id'] == movie_id]
movie_id = choosed_movie.iloc[0,0]
title = choosed_movie.iloc[0,1]
genre = choosed_movie.iloc[0,2]

print(f"선택한 유저 아이디 : {user_id}")
print(f"선택한 영화 아이디 : {movie_id}")
print(f"선택한 영화 제목 : {title}")
print(f"선택한 영화 장르 : {genre}")

user, amollang = user_to_idx[user_id], movie_to_idx[movie_id]
user_vector, amollang_vector = als_model.user_factors[user], als_model.item_factors[amollang]

correlation_score = np.dot(user_vector, amollang_vector)
print(f"선택한 유저와 영화 사이의 상관관계 점수 : {correlation_score:.2f}")

선택한 유저 아이디 : 6041
선택한 영화 아이디 : 3957
선택한 영화 제목 : amollang(2026)
선택한 영화 장르 : Comedy
선택한 유저와 영화 사이의 상관관계 점수 : 0.37


- 내가 선택한 영화의 선호도

In [28]:
user_id = 6041
movie_id, title, genre = choose_random_movie(movies)
print(f"선택한 유저 아이디 : {user_id}")
print(f"선택한 영화 아이디 : {movie_id}")
print(f"선택한 영화 제목 : {title}")
print(f"선택한 영화 장르 : {genre}")

user, amollang = user_to_idx[user_id], movie_to_idx[movie_id]
user_vector, amollang_vector = als_model.user_factors[user], als_model.item_factors[amollang]

correlation_score = np.dot(user_vector, amollang_vector)
print(f"선택한 유저와 영화 사이의 상관관계 점수 : {correlation_score:.2f}")


choosed_movie = movies.loc[movies['movie_id'] == movie_id]

선택한 유저 아이디 : 6041
선택한 영화 아이디 : 483
선택한 영화 제목 : King of the Hill (1993)
선택한 영화 장르 : Drama
선택한 유저와 영화 사이의 상관관계 점수 : 0.02


- 다른 영화의 선호도

# 7. 내가 좋아하는 영화와 비슷한 영화 추천

In [29]:
def idx_to_movie_name(df_movies, index_list):
    for i, index in enumerate(index_list):
        choosed_movie = df_movies.iloc[index]
        movie_id = choosed_movie[0]
        title = choosed_movie[1]
        genre = choosed_movie[2]

        print(f"{i}번째 영화")
        print(f"유사한 영화 아이디 : {movie_id}")
        print(f"유사한 영화 제목 : {title}")
        print(f"유사한 영화 장르 : {genre}")
        print("\n")

In [30]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

def get_similar_artist(num_recommendation, df_movies, idx_to_movie):
    movie_id, title, genre = choose_random_movie(df_movies)
    print(f"선택한 영화 아이디 : {movie_id}")
    print(f"선택한 영화 제목 : {title}")
    print(f"선택한 영화 장르 : {genre}")
    print("\n")
    
    similar_movie = als_model.similar_items(movie_id, N=num_recommendation)
    similar_movie_index = similar_movie[0][1:]

    movie_recommendation = [idx_to_movie[i] for i in similar_movie_index]
    idx_to_movie_name(df_movies, movie_recommendation)

In [31]:
get_similar_artist(15, movies, idx_to_movie)

선택한 영화 아이디 : 2314
선택한 영화 제목 : Beloved (1998)
선택한 영화 장르 : Drama


0번째 영화
유사한 영화 아이디 : 2503
유사한 영화 제목 : Apple, The (Sib) (1998)
유사한 영화 장르 : Drama


1번째 영화
유사한 영화 아이디 : 3852
유사한 영화 제목 : Tao of Steve, The (2000)
유사한 영화 장르 : Comedy


2번째 영화
유사한 영화 아이디 : 2922
유사한 영화 제목 : Hang 'em High (1967)
유사한 영화 장르 : Western


3번째 영화
유사한 영화 아이디 : 3044
유사한 영화 제목 : Dead Again (1991)
유사한 영화 장르 : Mystery|Romance|Thriller


4번째 영화
유사한 영화 아이디 : 2325
유사한 영화 제목 : Orgazmo (1997)
유사한 영화 장르 : Comedy


5번째 영화
유사한 영화 아이디 : 2059
유사한 영화 제목 : Parent Trap, The (1998)
유사한 영화 장르 : Children's|Drama


6번째 영화
유사한 영화 아이디 : 2509
유사한 영화 제목 : Eight Days a Week (1997)
유사한 영화 장르 : Comedy


7번째 영화
유사한 영화 아이디 : 3014
유사한 영화 제목 : Bustin' Loose (1981)
유사한 영화 장르 : Comedy


8번째 영화
유사한 영화 아이디 : 1352
유사한 영화 제목 : Albino Alligator (1996)
유사한 영화 장르 : Crime|Thriller


9번째 영화
유사한 영화 아이디 : 1717
유사한 영화 제목 : Scream 2 (1997)
유사한 영화 장르 : Horror|Thriller


10번째 영화
유사한 영화 아이디 : 2826
유사한 영화 제목 : 13th Warrior, The (1999)
유사한 영화 장르 : Action|Horror|Thriller

# 8. 내가 좋아할만한 영화 추천

In [34]:
user_id = user_to_idx[choose_random_user(ratings)]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommendation_index = als_model.recommend(user_id, csr_data[user_id], N=15, filter_already_liked_items=True)
movie_recommendation_index = movie_recommendation_index[0]

In [36]:
movie_id = [idx_to_movie[index] for index in movie_recommendation_index]
idx_to_movie_name(movies, movie_id)

0번째 영화
유사한 영화 아이디 : 2371
유사한 영화 제목 : Fletch (1985)
유사한 영화 장르 : Comedy


1번째 영화
유사한 영화 아이디 : 1687
유사한 영화 제목 : Jackal, The (1997)
유사한 영화 장르 : Action|Thriller


2번째 영화
유사한 영화 아이디 : 2493
유사한 영화 제목 : Harmonists, The (1997)
유사한 영화 장르 : Drama


3번째 영화
유사한 영화 아이디 : 12
유사한 영화 제목 : Dracula: Dead and Loving It (1995)
유사한 영화 장르 : Comedy|Horror


4번째 영화
유사한 영화 아이디 : 1992
유사한 영화 제목 : Child's Play 2 (1990)
유사한 영화 장르 : Horror


5번째 영화
유사한 영화 아이디 : 2740
유사한 영화 제목 : Kindred, The (1986)
유사한 영화 장르 : Horror


6번째 영화
유사한 영화 아이디 : 504
유사한 영화 제목 : No Escape (1994)
유사한 영화 장르 : Action|Sci-Fi


7번째 영화
유사한 영화 아이디 : 1623
유사한 영화 제목 : Wishmaster (1997)
유사한 영화 장르 : Horror


8번째 영화
유사한 영화 아이디 : 346
유사한 영화 제목 : Backbeat (1993)
유사한 영화 장르 : Drama|Musical


9번째 영화
유사한 영화 아이디 : 444
유사한 영화 제목 : Even Cowgirls Get the Blues (1993)
유사한 영화 장르 : Comedy|Romance


10번째 영화
유사한 영화 아이디 : 2390
유사한 영화 제목 : Little Voice (1998)
유사한 영화 장르 : Comedy


11번째 영화
유사한 영화 아이디 : 1538
유사한 영화 제목 : Second Jungle Book: Mowgli & Baloo, The (1997)
유사한 영

In [38]:
movie_idx = movie_to_idx[2371]
explain = als_model.explain(user_id, csr_data, itemid=movie_idx)

In [40]:
correlation_list = [(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[(2359, 0.03515240456402485),
 (587, 0.02438797745062124),
 (141, 0.016615751160191723),
 (357, 0.016182568579184212),
 (52, 0.015241729262914682),
 (852, 0.01407548928255472),
 (3809, 0.01327744187530803),
 (348, 0.010247280165900215),
 (1957, 0.009265439766394955),
 (3129, 0.007059931647123518)]

In [ ]:
movie_id = [idx_to_movie[index] for index in correlation_list]
idx_to_movie_name(movies, movie_id)

- 추천에 기여한 영화들의 정보